In [13]:
import pandas as pd

# Load the summerOly_athletes.csv dataset
athletes_path = './data/summerOly_athletes.csv'
athletes_df = pd.read_csv(athletes_path)

# Remove duplicate entries for the same athlete (Name), Year, and NOC
unique_athletes_df = athletes_df.drop_duplicates(subset=['Year', 'NOC', 'Name'])

# Group by Year and NOC to calculate the total number of unique athletes
total_athletes_feature = unique_athletes_df.groupby(['Year', 'NOC']).size()

# Convert to a DataFrame for further processing
total_athletes_feature = total_athletes_feature.reset_index(name='Total_Athletes')

# Use this feature directly in the linear regression workflow
total_athletes_feature.head()


,Year,NOC,Total_Athletes
0,1896,AUS,1
1,1896,AUT,3
2,1896,DEN,3
3,1896,FRA,12
4,1896,GBR,10


In [15]:
# Identify the most frequently occurring Team for each NOC
most_common_team_per_noc = (
    athletes_df.groupby('NOC')['Team']
    .agg(lambda x: x.mode()[0])  # Mode gives the most frequent value
    .reset_index()
    .rename(columns={'Team': 'Most_Frequent_Team'})
)

# Replace NOC with its most frequent Team value
athletes_df['NOC'] = athletes_df['NOC'].map(
    most_common_team_per_noc.set_index('NOC')['Most_Frequent_Team']
)

# Remove duplicate entries for the same athlete (Name), Year, and NOC
unique_athletes_df = athletes_df.drop_duplicates(subset=['Year', 'NOC', 'Name'])

# Group by Year and NOC to calculate the total number of unique athletes
total_athletes_by_noc = unique_athletes_df.groupby(['Year', 'NOC']).size().reset_index(name='Total_Athletes')

# Save the resulting DataFrame to a CSV file
output_path = 'total_unique_athletes_by_noc.csv'  # Change path if needed
total_athletes_by_noc.to_csv(output_path, index=False)

# Display the first few rows for verification
total_athletes_by_noc.head()


,Year,NOC,Total_Athletes
0,1896,Australia,1
1,1896,Austria,3
2,1896,Denmark,3
3,1896,France,12
4,1896,Germany,19


In [16]:
import pandas as pd

# Load the summerOly_medal_counts.csv dataset
medal_counts_path = './data/summerOly_medal_counts.csv'
medal_counts_df = pd.read_csv(medal_counts_path)

# Generate a complete list of all unique years and NOCs
all_years = medal_counts_df['Year'].unique()
all_nocs = medal_counts_df['NOC'].unique()

# Create a DataFrame with all possible combinations of Year and NOC
all_combinations = pd.MultiIndex.from_product(
    [all_years, all_nocs], names=['Year', 'NOC']
).to_frame(index=False)

# Merge the original medal counts with the full combination DataFrame
complete_medal_counts = pd.merge(
    all_combinations,
    medal_counts_df,
    on=['Year', 'NOC'],
    how='left'
)

# Fill missing values in the medal counts with 0
complete_medal_counts.fillna({'Gold': 0, 'Silver': 0, 'Bronze': 0, 'Total': 0}, inplace=True)

# Ensure all counts are integers
complete_medal_counts[['Gold', 'Silver', 'Bronze', 'Total']] = complete_medal_counts[['Gold', 'Silver', 'Bronze', 'Total']].astype(int)

# Save the resulting DataFrame to a CSV file
output_path = 'complete_medal_counts.csv'
complete_medal_counts.to_csv(output_path, index=False)

# Display the first few rows for verification
complete_medal_counts.head()


,Year,NOC,Rank,Gold,Silver,Bronze,Total
0,1896,United States,1.0,11,7,2,20
1,1896,Greece,2.0,10,18,19,47
2,1896,Germany,3.0,6,5,2,13
3,1896,France,4.0,5,4,2,11
4,1896,Great Britain,5.0,2,3,2,7


In [19]:
import pandas as pd

# Load the summerOly_hosts.csv dataset
hosts_path = './data/summerOly_hosts.csv'
hosts_df = pd.read_csv(hosts_path)

# Strip any leading/trailing whitespace from column names
hosts_df.columns = hosts_df.columns.str.strip()

# Rename columns if necessary to standardize (e.g., "ï»¿Year" becomes "Year")
hosts_df.rename(columns={'ï»¿Year': 'Year'}, inplace=True)

# Extract only the country (or NOC) part from the Host column
hosts_df['host_NOC'] = hosts_df['Host'].str.extract(r',\s*(.*)')[0]

# Handle years with no host
# Add entries for years with no host
no_host_years = [1916, 1940, 1944, 2020]
for year in no_host_years:
    hosts_df = pd.concat([hosts_df, pd.DataFrame({'Year': [year], 'host_NOC': ['No Host']})])

# Replace blank values or any inconsistent entries with "No Host"
hosts_df['host_NOC'] = hosts_df['host_NOC'].fillna('No Host')
hosts_df['host_NOC'] = hosts_df['host_NOC'].replace('', 'No Host')

# Remove duplicate rows based on 'Year' to ensure one entry per year
hosts_df = hosts_df.drop_duplicates(subset=['Year'], keep='first')

# Ensure all years are included for completeness
hosts_df.sort_values(by='Year', inplace=True)

# Save the modified DataFrame to a new CSV file
output_path = 'summerOly_hosts_with_host_NOC_cleaned.csv'
hosts_df.to_csv(output_path, index=False)

# Display the resulting DataFrame for verification
hosts_df.head()


,Year,Host,host_NOC
0,1896,"Athens, Greece",Greece
1,1900,"Paris, France",France
2,1904,"St. Louis, United States",United States
3,1908,"London, United Kingdom",United Kingdom
4,1912,"Stockholm, Sweden",Sweden


In [20]:
import pandas as pd

# Load the total_unique_athletes.csv
athletes_path = 'total_unique_athletes_by_noc.csv'
total_athletes_df = pd.read_csv(athletes_path)

# Load the medal counts dataset (to get all possible NOCs and Years)
medal_counts_path = './data/summerOly_medal_counts.csv'
medal_counts_df = pd.read_csv(medal_counts_path)

# Create a complete list of all unique Years and NOCs
all_years = medal_counts_df['Year'].unique()
all_nocs = medal_counts_df['NOC'].unique()
all_combinations = pd.MultiIndex.from_product([all_years, all_nocs], names=['Year', 'NOC']).to_frame(index=False)

# Merge the total_athletes_df with the complete list of Years and NOCs
complete_total_athletes_df = pd.merge(
    all_combinations,
    total_athletes_df,
    on=['Year', 'NOC'],
    how='left'
)

# Fill missing values with 0 for countries that did not participate
complete_total_athletes_df['Total_Athletes'] = complete_total_athletes_df['Total_Athletes'].fillna(0).astype(int)

# Save the resulting DataFrame to a CSV file
output_path = 'complete_total_unique_athletes_by_noc.csv'
complete_total_athletes_df.to_csv(output_path, index=False)

# Display the first few rows for verification
complete_total_athletes_df.head()


,Year,NOC,Total_Athletes
0,1896,United States,14
1,1896,Greece,102
2,1896,Germany,19
3,1896,France,12
4,1896,Great Britain,10


In [21]:
import pandas as pd

# Load the complete total unique athletes dataset
athletes_path = 'complete_total_unique_athletes_by_noc.csv'
total_athletes_df = pd.read_csv(athletes_path)

# Load the complete total medals dataset
medals_path = 'complete_medal_counts.csv'
total_medals_df = pd.read_csv(medals_path)

# Keep only necessary columns from the medals dataset and rename 'Total' to 'Total_Medal_Count'
total_medals_df = total_medals_df[['Year', 'NOC', 'Total']].rename(columns={'Total': 'Total_Medal_Count'})

# Merge the two datasets on Year and NOC
merged_df = pd.merge(
    total_athletes_df,
    total_medals_df,
    on=['Year', 'NOC'],
    how='outer'
).fillna(0)

# Ensure numeric columns are of integer type after filling NaNs
numeric_columns = ['Total_Athletes', 'Total_Medal_Count']
merged_df[numeric_columns] = merged_df[numeric_columns].astype(int)

# Save the resulting concatenated DataFrame to a CSV file
output_path = 'concatenated_athletes_medals.csv'
merged_df.to_csv(output_path, index=False)

# Display the first few rows for verification
merged_df.head()


,Year,NOC,Total_Athletes,Total_Medal_Count
0,1896,Afghanistan,0,0
1,1896,Albania,0,0
2,1896,Algeria,0,0
3,1896,Argentina,0,0
4,1896,Argentina,0,0


In [22]:
import pandas as pd

# Load the concatenated athletes and medals dataset
athletes_medals_path = 'concatenated_athletes_medals.csv'
merged_df = pd.read_csv(athletes_medals_path)

# Load the cleaned hosts dataset
hosts_path = 'summerOly_hosts_with_host_NOC_cleaned.csv'
hosts_df = pd.read_csv(hosts_path)

# Merge the hosts dataset to include host_NOC
merged_with_hosts = pd.merge(
    merged_df,
    hosts_df[['Year', 'host_NOC']],
    on='Year',
    how='left'
)

# Create the host_status feature
merged_with_hosts['host_status'] = (merged_with_hosts['host_NOC'] == merged_with_hosts['NOC']).astype(int)

# Handle missing values
merged_with_hosts['Total_Athletes'] = merged_with_hosts['Total_Athletes'].fillna(0).astype(int)
merged_with_hosts['Total_Medal_Count'] = merged_with_hosts['Total_Medal_Count'].fillna(0).astype(int)
merged_with_hosts['host_status'] = merged_with_hosts['host_status'].fillna(0).astype(int)


# Drop the host_NOC column as it's no longer needed
merged_with_hosts.drop(columns=['host_NOC'], inplace=True)

# Save the resulting DataFrame to a CSV file
output_path = 'concatenated_with_host_status.csv'
merged_with_hosts.to_csv(output_path, index=False)

# Display the first few rows for verification
merged_with_hosts.head()


,Year,NOC,Total_Athletes,Total_Medal_Count,host_status
0,1896,Afghanistan,0,0,0
1,1896,Albania,0,0,0
2,1896,Algeria,0,0,0
3,1896,Argentina,0,0,0
4,1896,Argentina,0,0,0
